# Development Notebook of Benji

...basically a nice way to run my code ;) In its own notebook for version management reasons.

Note to find right camera:

```
v4l2-ctl --list-devices
```
--> numbers correspond to numbers that OpenCV uses


## Demo: Capturing via Threading

This block captures images of two cameras in a sepearte thread and displays them in the main thread. Press `q` to quit.

In [ ]:
import threading
from queue import Queue
import cv2

from src.frame_container import FrameContainer
from src.camera_capture import CameraSetup
from src.camera_capture import capture_frames
from src.camera_capture import get_frame_from_queue

### PARAMETERS ###

keyword = "bottle"
"""Keyword inputed by user = what robot needs to find"""

### FUNCTIONS ###

"""
Note

This block is to be removed eventually. Functions and classes should all live in files.
"""

def process_frames(window_name : str, queue_frame: Queue, event_stop: threading.Event):
    while True:
        frame_container = get_frame_from_queue(queue_frame)
        if frame_container is None:
            continue
        cv2.imshow(window_name, frame_container.get_raw_info_frame())
        if cv2.waitKey(1) & 0xFF == ord('q'): #stop if the q button is pressed
            event_stop.set()
            return


### SCRIPT ###
print(f"Searching for '{keyword}'...")

# set up camera capturing
window_name = "Raw Capturing"
camera_setup = CameraSetup(6,2) #capturing camera port 0 and camera port 1
queue_frame_caputure = Queue(maxsize=5) #capturing to a queue of length 5
event_stop_capture = threading.Event()

thread_capture = threading.Thread( #setting up thread
    target=capture_frames,
    args=(camera_setup, queue_frame_caputure, event_stop_capture)
)
thread_capture.start()
print("Started camera capturing.")

# process the frames

try:
    #process the captured frames
    process_frames(window_name=window_name, queue_frame=queue_frame_caputure, event_stop=event_stop_capture)
finally:
    # Stop capture thread
    event_stop_capture.set()
    thread_capture.join()
    cv2.destroyAllWindows()

## Playground: Single Image

Playground to process a single image (left). A thread that captures the images is spawned and then a single image taken

In [ ]:
import threading
from queue import Queue
import cv2

from src.camera_capture import CameraSetup
from src.camera_capture import capture_frames
from src.camera_capture import get_frame_from_queue

### PARAMETERS ###

camera_setup = CameraSetup(0,1) #capturing camera port 0 and camera port 1
queue_frame_caputure = Queue(maxsize=1) #capturing to a queue of length 1 = always have most up to date image
event_stop_capture = threading.Event()
### SCRIPT ###

thread_capture = threading.Thread( #setting up thread
    target=capture_frames,
    args=(camera_setup, queue_frame_caputure, event_stop_capture)
)
thread_capture.start()
print("Started camera capturing. Will not stop this queue!")

In [ ]:
import warnings


def process_single_image(queue_frame: Queue):
    """Function to process a single image
    
    Meant to play around with --> Yolo, depth estimation ;)"""

    # get a frame from the queue
    container = get_frame_from_queue(queue_frame=queue_frame)
    if container is None:
        warnings.warn("Could not get a container from the frame queue! Ensure that somebody fills that one up!")
        return
    # extract the left image
    image = container.frame_left

    ### HAVE FUN WIHT IMAGE ###

    print("So much fun!")

    # display image for 10s at most, press key to exit
    cv2.imshow("Single Image Test",image)
    cv2.waitKey(10000)
    cv2.destroyAllWindows()

    return


try:
    #process the captured frames
    process_single_image(queue_frame=queue_frame_caputure)
finally:
    cv2.destroyAllWindows()

In [8]:
### STOP THE CAPTURING ###
event_stop_capture.set()
thread_capture.join()
print("Stopped the image capturing thread!")

## Structure

In [ ]:
import threading
from queue import Queue
import cv2
import warnings
import time

from src.frame_container import FrameContainer
from src.camera_capture import CameraSetup
from src.camera_capture import capture_frames
from src.camera_capture import get_frame_from_queue

### PARAMETERS ###

keyword = "bottle"
"""Keyword inputed by user = what robot needs to find"""

### FUNCTIONS ###

"""
Note

This block is to be removed eventually. Functions and classes should all live in files.
"""

def process_frames(window_name : str, queue_frame: Queue, event_stop: threading.Event):
    while True:
        frame_container = get_frame_from_queue(queue_frame)
        if frame_container is None:
            continue
        cv2.imshow(window_name, frame_container.get_raw_info_frame())
        if cv2.waitKey(1) & 0xFF == ord('q'): #stop if the q button is pressed
            event_stop.set()
            return
        

def detect_objects(container: FrameContainer) -> FrameContainer:
    """Detects Objects in the frame container.
    
    Directly modifies the Frame Container (returns same container)"""
    time_start = time.time()

    warnings.warn("TODO implement this function. Right now this is just a dummy function.")

    time_end = time.time()
    print(f"[Object Detection] Execution time:\t{time_end - time_start:.6f} s")
    return container

def estimate_distance(container: FrameContainer) -> FrameContainer:
    """Estimates the distance to the best matched object in the frame container
    
    Directly modifies the Frame Container (returns same container)"""
    time_start = time.time()

    warnings.warn("TODO implement this function. Right now this is just a dummy function.")

    time_end = time.time()
    print(f"[Distance Estimation] Execution time:\t{time_end - time_start:.6f} s")
    return container

def evaluate_captured_frames(queue_captured: Queue, #src queue
                             queue_distance: Queue, #destination queue
                             event_stop: threading.Event,
                             keyword: str):
    """Evaluate captured frames and putting the good ones into a queue
    
    run in a thread"""

    while not event_stop.is_set():
        # load latest from queue
        frame_container = get_frame_from_queue(queue_frame=queue_frame_caputure)
        if frame_container is None:
            continue

        # do object detection
        frame_container = detect_objects(frame_container)

        # give matching rating
        frame_container.rate_matching(keyword)

        # if above certain threshold, do evaluate distance and put into closer choice queue
        if frame_container.matchings is not None \
            and frame_container.is_matching_significant():
            # calculate distance
            frame_container = estimate_distance(frame_container)

            # put these frames into a queueq
            if queue_frame_distance.full():
                queue_frame_distance.get()
            queue_frame_distance.put(frame_container)
    return



### SCRIPT ###
print(f"Searching for '{keyword}'...")

# set up camera capturing
window_name = "Raw Capturing"
camera_setup = CameraSetup(0,2) #capturing camera port 0 and camera port 1
queue_frame_caputure = Queue(maxsize=2) #capturing to a queue of length 1 --> always most recent image (sometimes errors if it is empty)
# set up matching
queue_frame_distance = Queue(maxsize=5) #queue for frames that have distance information

event_stop_capture = threading.Event()
thread_capture = threading.Thread( #setting up thread
    target=capture_frames,
    args=(camera_setup, queue_frame_caputure, event_stop_capture)
)

event_stop_evaluate = threading.Event()
thread_evaluate = threading.Thread(
    target=evaluate_captured_frames,
    args=(queue_frame_caputure,queue_frame_distance,event_stop_evaluate, keyword)
)

# start threads
thread_capture.start()
print("Started camera capturing.")
thread_evaluate.start()
print("Started evaluation thread...")

# now can do stuff with the frames in the distance queue
# (1) display
# (2) if have had high matching object with similar distance for a few frames --> call it the final candidate